<a href="https://colab.research.google.com/github/ericphamhung-gretel/demos/blob/main/data_designer_sdk_qa_pairs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📚 Data Designer SDK: Generate Diverse Q&A Pairs

This notebook demonstrates a general approach for extracting Q&A pairs from any source document
(e.g. text, markdown, or PDF files). The generated Q&A pairs can be used for:
- **Instruction Tuning:** Training models with clear, self-contained examples.
- **Retrieval-Augmented Generation (RAG):** Enhancing retrieval systems with precise and context-supported Q&A pairs.
- **Search and FAQ Systems:** Powering natural language query systems and documentation.

> **Note:** The [Data Designer](https://docs.gretel.ai/create-synthetic-data/gretel-data-designer-beta) functionality demonstrated in this notebook is currently in **Early Preview**. To access these features and run this notebook, please [join the waitlist](https://gretel.ai/navigator/data-designer#waitlist).

# 📘 Getting Started

First, let's install and import the required packages:

In [1]:
# Install required libraries

%%capture
!pip install -qq langchain unstructured[pdf] smart_open git+https://github.com/gretelai/gretel-python-client

In [2]:
# Configuration
# -------------
# Define your source document(s) and the number of Q&A pairs to generate.
# You can replace this with your own documents in PDF, markdown, or text formats.

DOCUMENT_LIST = ["https://www.americanexpress.com/content/dam/amex/us/rewards/membership-rewards/mr-terms-conditions-11.01.2024.pdf?mrlinknav=footer-tandc"]
NUM_QA_PAIRS = 10

In [3]:
# Document Processing
# ------------------
# The DocumentProcessor class handles loading and chunking source documents for RAG evaluation.
# We use langchain's RecursiveCharacterTextSplitter and unstructured.io for robust document parsing.

from typing import List, Union
from langchain.text_splitter import RecursiveCharacterTextSplitter
from unstructured.partition.auto import partition
from smart_open import open
import tempfile
import os

class DocumentProcessor:
    def __init__(self, chunk_size: int = 4192, chunk_overlap: int = 200):
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
        )

    def parse_document(self, uri: str) -> str:
        """Parse a single document from URI into raw text."""
        with open(uri, 'rb') as file:
            content = file.read()
            with tempfile.NamedTemporaryFile(delete=False) as temp_file:
                temp_file.write(content)
                temp_file.flush()
                elements = partition(temp_file.name)

        os.unlink(temp_file.name)
        return "\n\n".join([str(element) for element in elements])

    def process_documents(self, uris: Union[str, List[str]]) -> List[str]:
        """Process one or more documents into chunks for RAG evaluation."""
        if isinstance(uris, str):
            uris = [uris]

        all_chunks = []
        for uri in uris:
            text = self.parse_document(uri)
            chunks = self.text_splitter.split_text(text)
            all_chunks.extend(chunks)

        return all_chunks

In [4]:
# Data Models
# -----------
# Define Pydantic models for structured output generation:
# 1. QAPair: Schema for question-answer pairs
# 2. EvalMetrics: Schema for scoring generation quality

from pydantic import BaseModel, Field
from typing import Optional, Literal

class QAPair(BaseModel):
    question: str = Field(..., description="A clear and concise question derived from the context.")
    answer: str = Field(..., description="A detailed and accurate answer fully supported by the context.")
    reasoning: str = Field(..., description="A brief explanation of why this Q&A pair is valuable.")


class EvalMetrics(BaseModel):
    clarity: int = Field(..., description="Clarity of the question", ge=1, le=5)
    factual_accuracy: int = Field(..., description="Factual accuracy of the answer", ge=1, le=5)
    comprehensiveness: int = Field(..., description="Completeness of the answer", ge=1, le=5)
    reasoning_quality: int = Field(..., description="Quality of the provided reasoning", ge=1, le=5)

In [5]:
# Setup & Configure Data Designer
# --------------------------------
# Initialize the Data Designer with a custom system message to ensure that the generated
# Q&A pairs are high-quality, context-supported, and tailored for a variety of downstream applications.


from gretel_client.navigator import DataDesigner

# Process document chunks
processor = DocumentProcessor(chunk_size=4192, chunk_overlap=200)
chunks = processor.process_documents(DOCUMENT_LIST)

# Initialize Data Designer
designer = DataDesigner(
    api_key="prompt",
    model_suite="llama-3.x",  # or "apache-2.0" as needed
    endpoint="https://api.gretel.cloud",
    special_system_instructions="""\
You are an expert at generating high-quality, context-supported Q&A pairs.
Your output should be clear, concise, and factually correct.
Ensure that every question is self-contained and every answer is comprehensive and derived solely from the provided context.
"""
)

# Add Seed Columns for Controlled Diversity

# Context: Document chunks.
designer.add_categorical_seed_column(
    name="context",
    description="A chunk of text extracted from the source document.",
    values=chunks
)

# Difficulty with nested Sophistication.
designer.add_categorical_seed_column(
    name="difficulty",
    description="The overall difficulty level of the question.",
    values=["easy", "medium", "hard"],
    subcategories=[
        {
            "name": "sophistication",
            "values": {
                "easy": ["basic", "straightforward"],
                "medium": ["intermediate", "moderately complex"],
                "hard": ["advanced", "sophisticated"]
            }
        }
    ]
)

# Question Style: Tone and approach.
designer.add_categorical_seed_column(
    name="question_style",
    description="The style or tone of the question.",
    values=["factual", "exploratory", "analytical", "comparative"]
)

# Target Audience: Language complexity.
designer.add_categorical_seed_column(
    name="target_audience",
    description="The intended audience for the Q&A pair.",
    values=["novice", "intermediate", "expert"]
)

# Response Format: Expected answer style.
designer.add_categorical_seed_column(
    name="response_format",
    description="The format of the answer expected (e.g., short, detailed, step-by-step).",
    values=["short", "detailed", "step-by-step", "list"]
)

# Define Generation Template for Q&A Pairs
designer.add_generated_data_column(
    name="qa_pair",
    generation_prompt=(
        "\n{context}\n\n"
        "Based on the above context, generate a high-quality Q&A pair. The question should be clear and concise, "
        "and tailored for an audience at the '{target_audience}' level. The overall difficulty should be '{difficulty}', "
        "with a corresponding sophistication level (e.g., {sophistication}). The question style should be '{question_style}', "
        "and the answer should be provided in a '{response_format}' format. "
        "Ensure that the answer is factually accurate, fully derived from the context, and comprehensive. "
        "Additionally, include a brief explanation (reasoning) on why this Q&A pair is valuable.\n\n"
        "Output the result in JSON format as follows:\n"
        '{{\n'
        '  "question": "<your question>",\n'
        '  "answer": "<your answer>",\n'
        '  "reasoning": "<your explanation>"\n'
        '}}\n'
    ),
    data_config={
        "type": "structured",
        "params": {"model": QAPair}
    }
)

# Optional: Define Evaluation Template for Q&A Pairs
designer.add_generated_data_column(
    name="eval_metrics",
    llm_type="judge",
    generation_prompt=(
        "\n{context}\n\n"
        "For the above Q&A pair:\n"
        "{qa_pair}\n\n"
        "Evaluate the following criteria on a scale of 1 (lowest) to 5 (highest):\n"
        "1. Clarity: How clear and understandable is the question?\n"
        "2. Factual Accuracy: Is the answer fully supported by the context?\n"
        "3. Comprehensiveness: Does the answer cover all necessary details?\n"
        "4. Reasoning Quality: Is the provided reasoning well-justified and clear?\n\n"
        "Output the scores in JSON format."
    ),
    data_config={
        "type": "structured",
        "params": {"model": EvalMetrics}
    }
)

# Preview a Sample of 10 Generated Records
preview = designer.generate_dataset_preview()
# preview.display_sample_record()


[20:26:25] [INFO] 🦜 Using llama-3.x model suite
Gretel API Key: ··········
Logged in as eric.phamhung@gretel.ai ✅
[20:26:31] [INFO] 🚀 Generating dataset preview
[20:26:31] [INFO] 📥 Step 1: Load data seeds
[20:26:32] [INFO] 🎲 Step 2: Sample data seeds
[20:26:32] [INFO] 🦜 Step 3: Generate column from template >> generating qa pair
[20:26:54] [INFO] 🦜 Step 4: Generate column from template >> generating eval metrics
[20:27:14] [INFO] 👀 Your dataset preview is ready for a peek!


In [6]:
from rich.console import Console
from rich.table import Table
from collections import Counter
import pandas as pd

def generate_qa_report(df):
    console = Console()
    categories = ['difficulty', 'sophistication', 'question_style', 'target_audience', 'response_format']

    console.print("\n[bold blue]🧠 QA Pair Generation Report[/bold blue]", justify="center")
    console.print("=" * 80, justify="center")
    console.print(f"\n[bold]Total QA Pairs:[/bold] {len(df)}")

    for category in categories:
        if category in df.columns:
            # Only count non-empty values
            counts = Counter(x for x in df[category] if pd.notna(x) and x != '')
            if not counts:
                continue

            table = Table(title=f"\n{category.title()} Distribution")
            table.add_column("Category", style="cyan")
            table.add_column("Count", justify="right")
            table.add_column("Percentage", justify="right")

            total = sum(counts.values())
            for value, count in sorted(counts.items()):
                percentage = (count / total) * 100
                table.add_row(str(value), str(count), f"{percentage:.1f}%")

            console.print(table)

    if any(col.startswith('eval_metrics.') for col in df.columns):
        metrics_table = Table(title="\nQuality Metrics Summary")
        metrics_table.add_column("Metric")
        metrics_table.add_column("Average Score", justify="right")

        metrics = ['clarity', 'factual_accuracy', 'comprehensiveness', 'reasoning_quality']
        for metric in metrics:
            col = f'eval_metrics.{metric}'
            if col in df.columns:
                scores = df[col].dropna()
                if len(scores) > 0:
                    avg_score = scores.mean()
                    metrics_table.add_row(
                        metric.replace('_', ' ').title(),
                        f"{avg_score:.2f}/5.00"
                    )

        console.print(metrics_table)

generate_qa_report(preview.dataset)

🧠 QA Pair Generation Report

================================================================================

Total QA Pairs: 10

                                 
     Difficulty Distribution     
┏━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Category ┃ Count ┃ Percentage ┃
┡━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│ easy     │     2 │      20.0% │
│ hard     │     3 │      30.0% │
│ medium   │     5 │      50.0% │
└──────────┴───────┴────────────┘

                                           
        Sophistication Distribution        
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Category           ┃ Count ┃ Percentage ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│ advanced           │     2 │      20.0% │
│ intermediate       │     3 │      30.0% │
│ moderately complex │     2 │      20.0% │
│ sophisticated      │     1 │      10.0% │
│ straightforward    │     2 │      20.0% │
└────────────────────┴───────┴────────────┘

                                    
    Question_Style Distribution     
┏━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Category    ┃ Count ┃ Percentage ┃
┡━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│ analytical  │     4 │      40.0% │
│ comparative │     2 │      20.0% │
│ exploratory │     2 │      20.0% │
│ factual     │     2 │      20.0% │
└─────────────┴───────┴────────────┘

                                     
    Target_Audience Distribution     
┏━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Category     ┃ Count ┃ Percentage ┃
┡━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│ expert       │     2 │      20.0% │
│ intermediate │     3 │      30.0% │
│ novice       │     5 │      50.0% │
└──────────────┴───────┴────────────┘

                                     
    Response_Format Distribution     
┏━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Category     ┃ Count ┃ Percentage ┃
┡━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│ detailed     │     2 │      20.0% │
│ list         │     4 │      40.0% │
│ short        │     2 │      20.0% │
│ step-by-step │     2 │      20.0% │
└──────────────┴───────┴────────────┘

In [7]:
# View the preview dataset as a dataframe
preview.dataset

,context,difficulty,sophistication,question_style,target_audience,response_format,qa_pair,eval_metrics
0,If an airline loyalty program stops participat...,medium,moderately complex,factual,novice,detailed,"{""question"": ""Can I convert points back into M...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
1,If you don't make a timely payment of the mini...,hard,sophisticated,exploratory,novice,step-by-step,"{""question"": ""If I don't make a timely payment...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
2,The list of eligible charges can change from t...,medium,intermediate,analytical,intermediate,short,"{""question"": ""Can a second named Checking Acco...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
3,Welcome to Membership Rewards!\n\nLast Updated...,easy,straightforward,factual,intermediate,short,"{""question"": ""What happens if the Membership R...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
4,11\n\nLimitation on Arbitration\n\nArbitration...,medium,moderately complex,analytical,novice,list,"{""question"": ""What are the conditions under wh...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
5,American Express ADR c/o CT Corporation System...,easy,straightforward,analytical,intermediate,list,"{""question"": ""What are the options for resolvi...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
6,Welcome to Membership Rewards!\n\nLast Updated...,medium,intermediate,comparative,expert,step-by-step,"{""question"": ""How do the redemption options fo...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
7,arbitration organization and all parties in wr...,hard,advanced,exploratory,novice,list,"{""question"": ""What are the steps to resolve a ...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
8,Business Purchase Account\n\nNo\n\n$40\n\nNo\n...,hard,advanced,comparative,novice,detailed,"{""question"": ""What is the key difference in th...","{""clarity"": 4, ""factual_accuracy"": 5, ""compreh..."
9,Business Purchase Account\n\nNo\n\n$40\n\nNo\n...,medium,intermediate,analytical,expert,list,"{""question"": ""What are the eligibility require...","{""clarity"": 4, ""factual_accuracy"": 4, ""compreh..."


In [8]:
preview.display_sample_record(3)

                                             Categorical Seed Columns                                              
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name                ┃ Value                                                                                     ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ context             │ Welcome to Membership Rewards!                                                            │
│                     │                                                                                           │
│                     │ Last Updated: November 2024                                                               │
│                     │                                                                                           │
│                     │ Welcome to the Membership Rewards® Program (Rewards Program)! These Terms and Conditions  │
│                     │ are the agreement between you and the Rewards Program owner and operator, American        │
│                     │ Express Travel Related Services Company, Inc. These Terms and Conditions will help you    │
│                     │ understand how the Rewards Program works.                                                 │
│                     │                                                                                           │
│                     │ These Terms and Conditions relate only to the Rewards Program. For terms and conditions   │
│                     │ related to a specific product, please refer to the separate document(s) that you received │
│                     │ upon enrollment.                                                                          │
│                     │                                                                                           │
│                     │ We may make changes to the Rewards Program, and we may add to and/or change these Terms   │
│                     │ and Conditions at any time.                                                               │
│                     │                                                                                           │
│                     │ For example, we could:                                                                    │
│                     │                                                                                           │
│                     │ Change the number of Membership Rewards® points (points) you can earn for eligible        │
│                     │ purchases • Change the number of points required to redeem                                │
│                     │                                                                                           │
│                     │ for rewards                                                                               │
│                     │                                                                                           │
│                     │ Change rewards options based on your enrolled product(s)                                  │
│                     │                                                                                           │
│                     │ Impose caps and/or fees on earning and/or using points                                    │
│                     │                                                                                           │
│                     │ Increase annual and/or other program fees • Cancel rewards and/or the Rewards Program     │
│                     │                                                                                           │
│                     │ If we cancel the Rewards Program, we'll notify you at least 90 days in advance. You'll be │
│                     │ able to earn or use points durin

In [9]:
df = preview.dataset
df

,context,difficulty,sophistication,question_style,target_audience,response_format,qa_pair,eval_metrics
0,If an airline loyalty program stops participat...,medium,moderately complex,factual,novice,detailed,"{""question"": ""Can I convert points back into M...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
1,If you don't make a timely payment of the mini...,hard,sophisticated,exploratory,novice,step-by-step,"{""question"": ""If I don't make a timely payment...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
2,The list of eligible charges can change from t...,medium,intermediate,analytical,intermediate,short,"{""question"": ""Can a second named Checking Acco...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
3,Welcome to Membership Rewards!\n\nLast Updated...,easy,straightforward,factual,intermediate,short,"{""question"": ""What happens if the Membership R...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
4,11\n\nLimitation on Arbitration\n\nArbitration...,medium,moderately complex,analytical,novice,list,"{""question"": ""What are the conditions under wh...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
5,American Express ADR c/o CT Corporation System...,easy,straightforward,analytical,intermediate,list,"{""question"": ""What are the options for resolvi...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
6,Welcome to Membership Rewards!\n\nLast Updated...,medium,intermediate,comparative,expert,step-by-step,"{""question"": ""How do the redemption options fo...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
7,arbitration organization and all parties in wr...,hard,advanced,exploratory,novice,list,"{""question"": ""What are the steps to resolve a ...","{""clarity"": 5, ""factual_accuracy"": 5, ""compreh..."
8,Business Purchase Account\n\nNo\n\n$40\n\nNo\n...,hard,advanced,comparative,novice,detailed,"{""question"": ""What is the key difference in th...","{""clarity"": 4, ""factual_accuracy"": 5, ""compreh..."
9,Business Purchase Account\n\nNo\n\n$40\n\nNo\n...,medium,intermediate,analytical,expert,list,"{""question"": ""What are the eligibility require...","{""clarity"": 4, ""factual_accuracy"": 4, ""compreh..."


In [10]:
# # Generate and Analyze Dataset
# # ---------------------------
# # Uncomment these lines to generate evaluation pairs, save them to JSONL,
# # and analyze the coverage and quality of the generated dataset.
# NUM_EVALS = 10
# batch_job = designer.submit_batch_workflow(num_records=NUM_EVALS)
# dataset = batch_job.fetch_dataset(wait_for_completion=True)

# generate_qa_report(dataset)

# dataset.to_json('qa_pairs.jsonl', orient='records', lines=True)


# Using OpenAI

In [11]:
from openai import OpenAI
from google.colab import userdata


In [12]:
client = OpenAI(
    base_url="https://litellm-proxy.dev.gretel.cloud/v1",
    api_key=userdata.get("GRETEL_API_KEY")
)

# [m.id for m in client.models.list()]

In [18]:
system_prompt = """
You are an expert at generating high-quality, context-supported Q&A pairs.
Your output should be clear, concise, and factually correct.
Ensure that every question is self-contained and every answer is comprehensive and derived solely from the provided context.
"""

user_prompt = f'''

"\n{chunks}\n\n"
"Based on the above context, generate {NUM_QA_PAIRS} high-quality Q&A pair. The question should be clear and concise, "
"Ensure that the answer is factually accurate, fully derived from the context, and comprehensive. "
"Additionally, include a brief explanation (reasoning) on why this Q&A pair is valuable.\n\n"
"Output the result in JSON format as follows:\n"
"{{\n"
"  "question": "<your question>",\n"
"  "answer": "<your answer>",\n"
"  "reasoning": "<your explanation>"\n"
"}}\n"


'''

In [19]:
completion = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": user_prompt
        }
    ]
)
print(completion.choices[0].message)




ChatCompletionMessage(content='{\n  "Q&A": [\n    {\n      "question": "What happens to your Membership Rewards points if American Express cancels your Rewards Program?",\n      "answer": "If American Express cancels the Rewards Program, they will notify you at least 90 days in advance. You will be able to earn or use points during that time, but certain rewards may be changed or canceled.",\n      "reasoning": "This information is valuable as it clarifies the process and timeframe if American Express decides to terminate the Rewards Program. It helps users understand how they can manage their points before the program ends."\n    },\n    {\n      "question": "Can you transfer Membership Rewards points to someone else\'s Rewards Account?",\n      "answer": "No, you cannot transfer Membership Rewards points to someone else\'s Rewards Account. Points do not belong to you and are not your property. You can\'t transfer them, sell them, or pass them on as part of a legal action.",\n      "r

In [20]:
print(completion.choices[0].message.content)

{
  "Q&A": [
    {
      "question": "What happens to your Membership Rewards points if American Express cancels your Rewards Program?",
      "answer": "If American Express cancels the Rewards Program, they will notify you at least 90 days in advance. You will be able to earn or use points during that time, but certain rewards may be changed or canceled.",
      "reasoning": "This information is valuable as it clarifies the process and timeframe if American Express decides to terminate the Rewards Program. It helps users understand how they can manage their points before the program ends."
    },
    {
      "question": "Can you transfer Membership Rewards points to someone else's Rewards Account?",
      "answer": "No, you cannot transfer Membership Rewards points to someone else's Rewards Account. Points do not belong to you and are not your property. You can't transfer them, sell them, or pass them on as part of a legal action.",
      "reasoning": "Understanding this restriction i